## 10 设计高效混合检索架构提升召回精度

### 1 混合检索的作用（Hybrid Search）

混合检索结合关键词匹配和语义搜索的优势，提供更准确、与上下文更相关的结果，进而让检索结果更全面。

例如，当搜索”苹果营养成分”时，混合检索既能找到包含”苹果”和”营养”关键词的文档，也能找到讨论”水果健康价值”等相关语义的内容，通过权重调整或重排序选出最优结果。

### 2 混合检索的架构（Hybrid Search Architecture）

混合检索架构通常包括关键词匹配、语义搜索和结果重排序等组件。

关键词匹配：使用 BM25 等传统的关键词匹配算法，快速筛选与查询关键词相关的文档。

语义搜索：利用向量空间模型（如 Word2Vec、BERT、GPT 等）将文档和查询转换为向量表示，通过余弦相似度计算文档与查询的相似性。

结果重排序：根据关键词匹配和语义搜索的结果，通过模型学习调整结果的排名顺序，提升与查询相关度较高的文档排名靠前。

### 3 混合检索的效果（Hybrid Search Effect）

混合检索的效果通常取决于架构设计、数据质量和模型训练。

较高的召回率：通过关键词匹配和语义搜索的组合，能够更全面地筛选相关文档，提升召回率。

较高的准确率：通过结果重排序模型，能够提升与查询相关的文档排名靠前的准确率，提升用户体验。

在复杂的信息检索任务中尤为有效。

### 4 以 Dify 为例，混合检索都有哪些常见的设置？

![Dify](./10/image.png)

## 5 使用 Milvus + LlamaIndex 构建 混合检索
### 5.1 安装依赖项


In [1]:
%pip install llama-index-vector-stores-milvus
%pip install llama-index-embeddings-openai
%pip install llama-index-llms-openai

Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 7.7 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.4/24.4 MB 25.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [llama-index-vector-stores-milvus]
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


### 5.2 启动 Milvus 服务器

docker compose up -d

注意：Milvus Standalone、Milvus Distributed 和 Zilliz Cloud 目前支持全文搜索，但 Milvus Lite 尚不支持全文搜索。

In [1]:
# 加载文档
from llama_index.core import SimpleDirectoryReader
documents = SimpleDirectoryReader("./10/docs/").load_data()
print("Example document:\n", documents[0])

Example document:
 Doc ID: adc4fe08-6c2f-4a74-b582-5a487eaaab84
Text: 第一回：灵根育孕源流出，心性修持大道生 <p>诗曰：</p><p>混沌未分天地乱，茫茫渺渺无人见。</p><p>自从盘古破鸿蒙，
开辟从兹清浊辨。</p><p>覆载群生仰至仁，发明万物皆成善。</p><p>欲知造化会元功，须看西游释厄传。</p><p>盖闻天地之数，有十
二万九千六百岁为一元。将一元分为十二会，乃子、丑、寅、卯、辰、巳、午、未、申、酉、戌、亥之十二支也。每会该一万八百岁。且就一日而论：子时得阳
气，而丑则鸡鸣；寅不通光，而卯则日出；辰时食后，而巳则挨排；日午天中，而未则西蹉；申时晡而日落酉；戌黄昏而人定亥。譬于大数，若到戌会之终，则
天地昏蒙而万物否矣。再去五千四百岁，交亥会之初，则当黑暗，而两间人物俱无矣，故曰混沌。又五千四百岁，亥会将终，贞下起元，近子之会，而复逐渐开
明。邵...


### 5.3 使用 BM25 执行混合搜索

In [5]:
from llama_index.vector_stores.milvus import MilvusVectorStore
from llama_index.core import StorageContext, VectorStoreIndex

URI = "http://localhost:19530"  # Milvus URI

vector_store = MilvusVectorStore(
    uri=URI,
    # token=TOKEN,
    dim=1536,  # vector dimension depends on the embedding model
    enable_sparse=True,  # enable the default full-text search using BM25
    overwrite=True,  # drop the collection if it already exists
)

storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)

2025-07-14 20:47:51,795 [DEBUG][_create_connection]: Created new connection using: async-http://localhost:19530 (async_milvus_client.py:599)
Sparse embedding function is not provided, using default.
Default sparse embedding function: BM25BuiltInFunction(input_field_names='text', output_field_names='sparse_embedding').


参数说明：

- dim (int, optional):Collections 的嵌入向量维度。
- enable_sparse (bool):用于启用或禁用稀疏嵌入。默认为假。




### 5.4 启用混合搜索

在查询阶段启用混合搜索，将vector_store_query_mode 设置为 "hybrid"。

开启后，将对语义搜索和全文搜索的搜索结果进行合并和 Rerankers。

In [6]:
import textwrap

query_engine = index.as_query_engine(
    vector_store_query_mode="hybrid", similarity_top_k=10
)
response = query_engine.query("孙悟空名字的由来？")

print(textwrap.fill(str(response), 100))

for idx, node in enumerate(response.source_nodes, 1):
        print(f"结果 {idx}: ")
        print(textwrap.fill(str(node.node.text), 100))
        print("\n")

孙悟空的名字由来是在他出生后被一位祖师所赐。祖师给他起名叫孙悟空，意味着他是天地生成的石猴，具有悟性和空灵之意。
结果 1: 
我在那里住？我来你面前扒柴挑菜！"悟空道："我儿子便胡说！你是认不得我了，我本是这两界山石匣中的大圣。你再认认看。"老者方才省悟道："你倒有些象他，但你是怎么得出来的？"悟空将菩萨劝善、令我等待唐僧揭
贴脱身之事，对那老者细说了一遍。老者却才下拜，将唐僧请到里面，即唤老妻与儿女都来相见，具言前事，个个欣喜。又命看茶，茶罢，问悟空道："大圣啊，你也有年纪了？"悟空道："你今年几岁了？"老者道："我痴长
一百三十岁了。"行者道："还是我重子重孙哩！我那生身的年纪，我不记得是几时，但只在这山脚下，已五百余年了。"老者道："是有，是有。我曾记得祖公公说，此山乃从天降下，就压了一个神猴。只到如今，你才脱体。
我那小时见你，是你头上有草，脸上有泥，还不怕你；如今脸上无了泥，头上无了草，却象瘦了些，腰间又苫了一块大虎皮，与鬼怪能差多少？"</p><p> 一家儿听得这般话说，都呵呵大笑。这老儿颇贤，即今安排斋饭
。饭后，悟空道："你家姓甚？"老者道："舍下姓陈。"三藏闻言，即下来起手道："老施主，与贫僧是华宗。"行者道："师父，你是唐姓，怎的和他是华宗？"三藏道："我俗家也姓陈，乃是唐朝海州弘农郡聚贤庄人氏。
我的法名叫做陈玄奘。只因我大唐太宗皇帝赐我做御弟三藏，指唐为姓，故名唐僧也。"那老者见说同姓，又十分欢喜。行者道："老陈，左右打搅你家。我有五百多年不洗澡了，你可去烧些汤来，与我师徒们洗浴洗浴，一发临
行谢你。"那老儿即令烧汤拿盆，掌上灯火。师徒浴罢，坐在灯前，行者道："老陈，还有一事累你，有针线借我用用。"那老儿道："有，有，有。"即教妈妈取针线来，递与行者。


结果 2: 
我老师父护住山泉，并不曾白送与人。你回去办将礼来，我好通报，不然请回，莫想莫想！"行者道："人情大似圣旨，你去说我老孙的名字，他必然做个人情，或者连井都送我也。" 那道人闻此言，只得进去通报，却见那真
仙抚琴，只待他琴终，方才说道："师父，外面有个和尚，口称是唐三藏大徒弟孙悟空，欲求落胎泉水，救他师父。"那真仙不听说便罢，一听得说个悟空名字，却就怒从心上起，恶向胆边生，急起身，下了琴床，脱了素服，换
上道衣，取一把如意钩子，跳出庵门，叫道："孙悟空何在？"行者转头，观见那真

In [42]:
# 导入必要的库
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
import textwrap

# 1. 加载文档
print("Loading documents...")
documents = SimpleDirectoryReader("./10/docs/").load_data()  # 替换为你的文档路径

# 2. 构建索引
print("Building index...")
index = VectorStoreIndex.from_documents(documents)

# 3. 设置混合检索参数
alpha = 0.1  # 权重参数：越大越偏向语义检索，越小偏向关键词检索
top_k = 10   # 返回前 top_k 个结果

retriever = index.as_retriever(
    retriever_mode="hybrid",
    similarity_top_k=top_k,
    alpha=alpha
)

# 4. 创建查询引擎
query_engine = RetrieverQueryEngine(retriever=retriever)

# 5. 执行查询
query = "孙悟空名字的由来？"
print(f"\nQuery: {query}")
response = query_engine.query(query)
print("\nResponse:")
print(textwrap.fill(str(response), 100))

# 6. 输出来源文档片段
print("\nSource Nodes:")
for idx, node in enumerate(response.source_nodes, 1):
    print(f"结果 {idx}: ")
    print(textwrap.fill(str(node.node.text), 100))
    print("\n")

Loading documents...
Building index...

Query: 孙悟空名字的由来？

Response:
孙悟空的名字由来是他在花果山上出生，被观音菩萨赐予法名“孙悟空”，意味着他悟透了菩提真妙理。

Source Nodes:
结果 1: 
我在那里住？我来你面前扒柴挑菜！"悟空道："我儿子便胡说！你是认不得我了，我本是这两界山石匣中的大圣。你再认认看。"老者方才省悟道："你倒有些象他，但你是怎么得出来的？"悟空将菩萨劝善、令我等待唐僧揭
贴脱身之事，对那老者细说了一遍。老者却才下拜，将唐僧请到里面，即唤老妻与儿女都来相见，具言前事，个个欣喜。又命看茶，茶罢，问悟空道："大圣啊，你也有年纪了？"悟空道："你今年几岁了？"老者道："我痴长
一百三十岁了。"行者道："还是我重子重孙哩！我那生身的年纪，我不记得是几时，但只在这山脚下，已五百余年了。"老者道："是有，是有。我曾记得祖公公说，此山乃从天降下，就压了一个神猴。只到如今，你才脱体。
我那小时见你，是你头上有草，脸上有泥，还不怕你；如今脸上无了泥，头上无了草，却象瘦了些，腰间又苫了一块大虎皮，与鬼怪能差多少？"</p><p> 一家儿听得这般话说，都呵呵大笑。这老儿颇贤，即今安排斋饭
。饭后，悟空道："你家姓甚？"老者道："舍下姓陈。"三藏闻言，即下来起手道："老施主，与贫僧是华宗。"行者道："师父，你是唐姓，怎的和他是华宗？"三藏道："我俗家也姓陈，乃是唐朝海州弘农郡聚贤庄人氏。
我的法名叫做陈玄奘。只因我大唐太宗皇帝赐我做御弟三藏，指唐为姓，故名唐僧也。"那老者见说同姓，又十分欢喜。行者道："老陈，左右打搅你家。我有五百多年不洗澡了，你可去烧些汤来，与我师徒们洗浴洗浴，一发临
行谢你。"那老儿即令烧汤拿盆，掌上灯火。师徒浴罢，坐在灯前，行者道："老陈，还有一事累你，有针线借我用用。"那老儿道："有，有，有。"即教妈妈取针线来，递与行者。


结果 2: 
我老师父护住山泉，并不曾白送与人。你回去办将礼来，我好通报，不然请回，莫想莫想！"行者道："人情大似圣旨，你去说我老孙的名字，他必然做个人情，或者连井都送我也。" 那道人闻此言，只得进去通报，却见那真
仙抚琴，只待他琴终，方才说道："师父，外面有个和尚，口称是唐三藏大徒弟孙悟空，欲求落胎泉水，救他师父。"那真仙不听说便罢，一听得说

In [43]:
# 导入必要的库
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
import textwrap

# 1. 加载文档
print("Loading documents...")
documents = SimpleDirectoryReader("./10/docs/").load_data()  # 替换为你的文档路径

# 2. 构建索引
print("Building index...")
index = VectorStoreIndex.from_documents(documents)

# 3. 设置混合检索参数
alpha = 0.9  # 权重参数：越大越偏向语义检索，越小偏向关键词检索
top_k = 10   # 返回前 top_k 个结果

retriever = index.as_retriever(
    retriever_mode="hybrid",
    similarity_top_k=top_k,
    alpha=alpha
)

# 4. 创建查询引擎
query_engine = RetrieverQueryEngine(retriever=retriever)

# 5. 执行查询
query = "孙悟空名字的由来？"
print(f"\nQuery: {query}")
response = query_engine.query(query)
print("\nResponse:")
print(textwrap.fill(str(response), 100))

# 6. 输出来源文档片段
print("\nSource Nodes:")
for idx, node in enumerate(response.source_nodes, 1):
    print(f"结果 {idx}: ")
    print(textwrap.fill(str(node.node.text), 100))
    print("\n")

Loading documents...
Building index...

Query: 孙悟空名字的由来？

Response:
孙悟空的名字由来是在他出生后被一位祖师赋予的法名。

Source Nodes:
结果 1: 
我在那里住？我来你面前扒柴挑菜！"悟空道："我儿子便胡说！你是认不得我了，我本是这两界山石匣中的大圣。你再认认看。"老者方才省悟道："你倒有些象他，但你是怎么得出来的？"悟空将菩萨劝善、令我等待唐僧揭
贴脱身之事，对那老者细说了一遍。老者却才下拜，将唐僧请到里面，即唤老妻与儿女都来相见，具言前事，个个欣喜。又命看茶，茶罢，问悟空道："大圣啊，你也有年纪了？"悟空道："你今年几岁了？"老者道："我痴长
一百三十岁了。"行者道："还是我重子重孙哩！我那生身的年纪，我不记得是几时，但只在这山脚下，已五百余年了。"老者道："是有，是有。我曾记得祖公公说，此山乃从天降下，就压了一个神猴。只到如今，你才脱体。
我那小时见你，是你头上有草，脸上有泥，还不怕你；如今脸上无了泥，头上无了草，却象瘦了些，腰间又苫了一块大虎皮，与鬼怪能差多少？"</p><p> 一家儿听得这般话说，都呵呵大笑。这老儿颇贤，即今安排斋饭
。饭后，悟空道："你家姓甚？"老者道："舍下姓陈。"三藏闻言，即下来起手道："老施主，与贫僧是华宗。"行者道："师父，你是唐姓，怎的和他是华宗？"三藏道："我俗家也姓陈，乃是唐朝海州弘农郡聚贤庄人氏。
我的法名叫做陈玄奘。只因我大唐太宗皇帝赐我做御弟三藏，指唐为姓，故名唐僧也。"那老者见说同姓，又十分欢喜。行者道："老陈，左右打搅你家。我有五百多年不洗澡了，你可去烧些汤来，与我师徒们洗浴洗浴，一发临
行谢你。"那老儿即令烧汤拿盆，掌上灯火。师徒浴罢，坐在灯前，行者道："老陈，还有一事累你，有针线借我用用。"那老儿道："有，有，有。"即教妈妈取针线来，递与行者。


结果 2: 
我老师父护住山泉，并不曾白送与人。你回去办将礼来，我好通报，不然请回，莫想莫想！"行者道："人情大似圣旨，你去说我老孙的名字，他必然做个人情，或者连井都送我也。" 那道人闻此言，只得进去通报，却见那真
仙抚琴，只待他琴终，方才说道："师父，外面有个和尚，口称是唐三藏大徒弟孙悟空，欲求落胎泉水，救他师父。"那真仙不听说便罢，一听得说个悟空名字，却就怒从心上起，恶向胆边生，急

### 5.5 加权平均（Weighted Average）

LlamaIndex 支持通过 alpha 参数对语义和关键词检索结果进行线性加权融合。
- alpha=1.0 表示完全依赖语义检索
- alpha=0.0 表示完全依赖关键词检索
- 中间值则表示两者加权融合

⚠️ 注意：这种加权方式是基于得分直接加权，并不是严格的归一化后加权。

## 6 扩展

支持混合检索的平台、工具及其稀疏嵌入方法

| 平台/工具                   | 示例稀疏嵌入方法                                      | 说明                                                                                     |
|----------------------------|-------------------------------------------------------|------------------------------------------------------------------------------------------|
| Qdrant + LlamaIndex          | BM42                                                  | 提供轻量级稀疏嵌入方法，并与语义向量结合实现高效的混合搜索 [[1]]                          |
| Milvus + LlamaIndex          | BM25BuiltInFunction                                  | 默认支持使用 BM25 对文本进行分词和权重计算，适用于关键词匹配场景 [[4]]                     |
| Milvus + LlamaIndex          | BGEM3SparseEmbeddingFunction                         | 使用 BGE-M3 模型生成稀疏嵌入，结合语义理解与关键词匹配 [[9]]                               |
| Milvus + LlamaIndex          | 自定义稀疏嵌入函数（如继承 BaseSparseEmbeddingFunction） | 支持用户自定义稀疏嵌入方法，如基于神经模型的稀疏表示 [[5]]                                 |
| Milvus                       | TF-IDF、BM25、SPLADEv2 稀疏向量                      | Milvus 原生支持多种稀疏向量格式，可与稠密向量同等处理，扩展了混合搜索功能 [[3]][[6]]       |
| BGE-M3                       | 内置词元权重（模拟 BM25 效果）                       | 不仅支持密集嵌入，还能生成类似 BM25 的稀疏表示，用于混合检索 [[9]]                         |                           |
| Elasticsearch                | Elastic Learned Sparse Encoder                       | 使用训练好的模型理解稀疏向量嵌入，并结合密集向量进行混合搜索 [[8]]                           |
| 自定义系统                   | BM25、TF-IDF、Sparse Transformers                    | 可集成任意稀疏嵌入，实现灵活的多源混合检索                                                 |